In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import pathlib
import PIL
from PIL import Image 
from PIL.ImageDraw import Draw

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Processing Data

In [2]:
collected_img = r'.\collectedimages' 
classes = os.listdir(collected_img)
num_classes = len(classes)

print(f'Classes : {classes}\nNum class : {num_classes}')

Classes : ['botolkaca', 'botolplastik', 'kaleng', 'kardus', 'karet', 'kertas', 'plastik', 'sedotan']
Num class : 8


In [10]:
datagen = ImageDataGenerator(
                rotation_range=10,
                width_shift_range=0.2,
                height_shift_range=0.2,
                rescale=1/255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest',
                validation_split=0.2   
                )

batch_size = 32

train_gene = datagen.flow_from_directory(
        collected_img,  
        target_size=(224, 224),  
        batch_size=batch_size,
        subset='training',
        classes=classes)

valid_gene = datagen.flow_from_directory(
        collected_img,
        target_size=(224, 224), 
        batch_size=batch_size, 
        subset='validation',
        classes=classes)

Found 3904 images belonging to 8 classes.
Found 972 images belonging to 8 classes.


## Model Creation

In [14]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

In [57]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x) 
x = Dropout(.5, seed=101)(x) 
x = Dense(2048,activation='relu')(x) 
x = Dropout(.5, seed=101)(x) 
preds = Dense(num_classes, activation='sigmoid')(x) 

model=Model(inputs=base_model.input, outputs=preds)

In [58]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [59]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [60]:
history = model.fit(train_gene, 
                    epochs = 20, 
                    steps_per_epoch = train_gene.samples // batch_size,
                    validation_data = valid_gene, 
                    validation_steps = valid_gene.samples // batch_size)

Epoch 1/20
122/122 [==============================] - 53s 423ms/step - loss: 1.2516 - accuracy: 0.5710 - val_loss: 1.4328 - val_accuracy: 0.4938
Epoch 2/20
122/122 [==============================] - 50s 409ms/step - loss: 0.6307 - accuracy: 0.7820 - val_loss: 1.4747 - val_accuracy: 0.5302
Epoch 3/20
122/122 [==============================] - 48s 397ms/step - loss: 0.4622 - accuracy: 0.8407 - val_loss: 1.4490 - val_accuracy: 0.5792
Epoch 4/20
122/122 [==============================] - 48s 396ms/step - loss: 0.3855 - accuracy: 0.8691 - val_loss: 1.5540 - val_accuracy: 0.5531
Epoch 5/20
122/122 [==============================] - 49s 398ms/step - loss: 0.3502 - accuracy: 0.8835 - val_loss: 1.5737 - val_accuracy: 0.5417
Epoch 6/20
122/122 [==============================] - 49s 401ms/step - loss: 0.3109 - accuracy: 0.8947 - val_loss: 1.7704 - val_accuracy: 0.5271
Epoch 7/20
  5/122 [>.............................] - ETA: 39s - loss: 0.2698 - accuracy: 0.8813

KeyboardInterrupt: 